In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tqdm.notebook import tqdm
import numpy as np
import pickle
import scipy.sparse as sp
import cupy as cp

In [ ]:
with open('/content/drive/MyDrive/nlp/vocabulary.pkl', 'rb') as f:
    vocabulary = pickle.load(f)
with open('/content/drive/MyDrive/nlp/word2ind.pkl', 'rb') as f1:
    word2ind = pickle.load(f1)
with open('/content/drive/MyDrive/nlp/ind2word.pkl', 'rb') as f2:
    ind2word = pickle.load(f2)

In [ ]:
x = sp.load_npz('/content/drive/MyDrive/cbow200/x_cbow.npz')
y = sp.load_npz('/content/drive/MyDrive/cbow200/y_cbow.npz')

In [ ]:
cp.random.seed(1)

In [ ]:
W0=cp.random.uniform(low=-0.8 ,high=0.8,size=(len(vocabulary,200)))
W1=cp.random.uniform(low=-0.8 ,high=0.8,size=(200,len(vocabulary)))

In [ ]:
def softmax(z):
    e_z = cp.exp(z)
    sum_e_z = cp.sum(e_z,axis=0,keepdims=True)
    return e_z/sum_e_z

In [ ]:
def forward(W0,W1,x):
    h=cp.dot(W0.T,x)
    v=cp.dot(W1.T,h)
    return h,softmax(v)

In [ ]:
def backprop(W0,W1,x,y_true,alpha):
    h,y_pred=forward(W0,W1,x)
    y_pred=cp.maximum(1e-8,y_pred)
    loss = - cp.sum(y_true*np.log(y_pred))/y_true.shape[1]
    l1 = cp.dot(W1 ,(y_pred - y_true))
    dW1 = cp.dot(h,(y_pred - y_true).T)
    dW0 = cp.dot(x,l1.T)
    W1 -= alpha * dW1
    W0 -= alpha * dW0
    return W0,W1,loss

In [ ]:
def minibatch(W0,W1,x,y,leng,epochs):
    W0x,W1x=W0,W1
    e=0
    while e<=epochs:
        info=""
        batchsize=100
        i=0
        while batchsize+i <= leng:    
            W0x,W1x,loss=backprop(W0x,W1x,cp.asarray(x[:,i:i+batchsize].toarray()),cp.asarray(y[:,i:i+batchsize].toarray()),0.03)
            if (i+batchsize)% 50000==0:
                info+=f"loss at epoch {e} with batch {i} - {i+batchsize} is {loss}\n"
                print(f"loss at epoch {e} with batch {i} - {i+batchsize} is {loss}")
          
            if i+batchsize == leng:
                info+=f"returned to batch of epoch {e} with loss at {i} - {i+batchsize} is {loss}\n"
                print(f"returned to batch of epoch {e} with loss at {i} - {i+batchsize} is {loss}")

                if not np.isnan(W0x[1,2]):
                    print(f"content w0,w1 saved for epoch {e} at {i+batchsize}")
                    np.save(f'/content/drive/MyDrive/cbow200/W0epoch{e}.npy',W0x)
                    np.save(f'/content/drive/MyDrive/cbow200/W1epoch{e}.npy',W1x)
                    with open('/content/drive/MyDrive/cbow200/loss.txt', 'a') as file:
                          file.write(info)

            if i+batchsize == 659400:
              batchsize=(leng-(batchsize+i))
          
            i+=100
      
        e+=1
    

In [ ]:
leng=x.shape[1]

In [ ]:
minibatch(W0,W1,x,y,leng,25)

loss at epoch 26 with batch 49900 - 50000 is 2.7676939403596803
loss at epoch 26 with batch 99900 - 100000 is 3.9408942324864142
loss at epoch 26 with batch 149900 - 150000 is 2.7427221139755575
loss at epoch 26 with batch 199900 - 200000 is 3.0027732379799774
loss at epoch 26 with batch 249900 - 250000 is 3.1340055002726035
loss at epoch 26 with batch 299900 - 300000 is 2.7138934877452265
loss at epoch 26 with batch 349900 - 350000 is 3.1570957344768056
loss at epoch 26 with batch 399900 - 400000 is 2.5952188132567016
loss at epoch 26 with batch 449900 - 450000 is 2.4136264543392527
loss at epoch 26 with batch 499900 - 500000 is 1.7271472766020293
loss at epoch 26 with batch 549900 - 550000 is 0.23507904759570686
loss at epoch 26 with batch 599900 - 600000 is 0.4504962518225017
loss at epoch 26 with batch 649900 - 650000 is 1.1867014752310323
returned to batch of epoch 26 with loss at 659400 - 659421 is 0.09537502227940738
content w0,w1 saved for epoch 26 at 659421
loss at epoch 27 wi